<h1>GRADUATION PROJECT<h1> STAGE 2

<ul><li>
   Data preprocessing and Cosine Similarity Algoritm.</li>

<li>This section includes data about title of a case study, synopsis, tags, industry, session notes. Industry and tags were manually added.
    </li>




In [64]:
import sklearn as sk
import pandas as pd
import numpy as np
import json
import random
from sklearn.model_selection import train_test_split #I need this to split the data
import math
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import re

In [65]:
df = pd.read_csv("luck2.csv",names=["data","industry","tag","tag2","tag3","sessionnote"],encoding="ISO-8859-1",delimiter=";")

In [66]:
df

,data,industry,tag,tag2,tag3,sessionnote
0,How Spotify organises work Spotify recently ad...,entertainment,music,spotify,NaN,Consider the site as a red brick road: move yo...
1,Designing a video creation platform. Designing...,health &psychology,platform,video,NaN,Customers are more likely to see things close ...
2,Applying user research to a small e-commerce w...,e-commerce,website,NaN,NaN,Steady and simple to use web interfaces assist...
3,Democratizing access to bike maps Cristiano sh...,travel&transportation,platform,bike maps,NaN,Abstain from making dead-end pages on sites. T...
4,Reimagine the future of television Abdus share...,entertainment,television,tv,NaN,Utilize common site patterns and interfaces
...,...,...,...,...,...,...
133,Rethinking an application against food waste a...,energy&earth,sustainability,waste,NaN,Customers are hesitant to utilize Help on your...
134,The essential guide to enterprise login and si...,tech,login,signup,NaN,Symbols should visually describe their purpose...
135,UX/UI Case Study |Â EcoAI As responsible resi...,energy&earth,sustainability,recycle,NaN,Icons should possibly be utilized when importa...
136,Medication Disposal UX/UIÂ Design Last week we...,health &psychology,sustainability,recycle,disposal,Put key data first. Customers start at the upp...


In [67]:
#make a seperate dataframes for every tag column from the main dataframe
df_t1=df["tag"]
df_t2=df["tag2"]
df_t3=df["tag3"]

In [68]:
df_t3

0           NaN
1           NaN
2           NaN
3           NaN
4           NaN
         ...   
133         NaN
134         NaN
135         NaN
136    disposal
137         NaN
Name: tag3, Length: 138, dtype: object

In [69]:
#this makes series to dataframe
df_t1 = pd.Series.to_frame(df_t1, name = "tags")
df_t2 = pd.Series.to_frame(df_t2, name = "tags")
df_t3 = pd.Series.to_frame(df_t3, name = "tags")

In [70]:
df_alltags = df_t1.copy() 

## Now all tags is going to be in one column. In the following steps, all tags from different dataframes will be combined.This is the first one.

In [71]:
df_alltags

,tags
0,music
1,platform
2,website
3,platform
4,television
...,...
133,sustainability
134,login
135,sustainability
136,sustainability


In [72]:
df_alltags.append(df_t2,ignore_index=True) #next step is to append all the tags

,tags
0,music
1,platform
2,website
3,platform
4,television
...,...
271,waste
272,signup
273,recycle
274,recycle


In [73]:
df_alltags=df_alltags.append(df_t2,ignore_index=True) #df_t2 is added

In [74]:
df_alltags=df_alltags.append(df_t3,ignore_index=True) #df_t3 is added 

In [75]:
df_alltags #now I have all the tags in one row in df_alltags. 414 tags but there are some dublicates

,tags
0,music
1,platform
2,website
3,platform
4,television
...,...
409,NaN
410,NaN
411,NaN
412,disposal


In [76]:
df_tags = df_alltags.groupby(["tags"]).sum().reset_index() #groupby to make it count one. There is no dublicated

## 414 tags decrease to 156 in the below. All unique tags are 156 rows from now on

In [77]:
df_tags

,tags
0,data visualization
1,AI
2,AR
3,HR
4,MacDonalds
...,...
151,vpn
152,waste
153,web page
154,website


In [78]:
for i in range (len(df)):
    df["tag"][i]= df_tags.where(df_tags == df["tag"][i]).dropna().index[0]  
    # df_tags are not repetitive anymore. df[tag] has all cells full. from the df_tag and put that number to df[tag]
    

In [79]:
# in tag2 and tag3, they are not fulled with tags. So not to get error because of NAN tags, here a NAN cell gets -1
for i in range (len(df)):
    try:
        if  math.isnan(df["tag2"][i]): #if it is not float or integer there will be an exception
            df["tag2"][i] = -1  # -1 is written to make Nan not have an index
    except:
        df["tag2"][i]= df_tags.where(df_tags == df["tag2"][i]).dropna().index[0]
    
    try:
        if math.isnan(df["tag3"][i]):
            df["tag3"][i] = -1
    except:
        df["tag3"][i]= df_tags.where(df_tags == df["tag3"][i]).dropna().index[0]

In [80]:
df #NAN tags become -1

,data,industry,tag,tag2,tag3,sessionnote
0,How Spotify organises work Spotify recently ad...,entertainment,95,128,-1,Consider the site as a red brick road: move yo...
1,Designing a video creation platform. Designing...,health &psychology,110,148,-1,Customers are more likely to see things close ...
2,Applying user research to a small e-commerce w...,e-commerce,154,-1,-1,Steady and simple to use web interfaces assist...
3,Democratizing access to bike maps Cristiano sh...,travel&transportation,110,20,-1,Abstain from making dead-end pages on sites. T...
4,Reimagine the future of television Abdus share...,entertainment,132,145,-1,Utilize common site patterns and interfaces
...,...,...,...,...,...,...
133,Rethinking an application against food waste a...,energy&earth,131,152,-1,Customers are hesitant to utilize Help on your...
134,The essential guide to enterprise login and si...,tech,76,121,-1,Symbols should visually describe their purpose...
135,UX/UI Case Study |Â EcoAI As responsible resi...,energy&earth,131,115,-1,Icons should possibly be utilized when importa...
136,Medication Disposal UX/UIÂ Design Last week we...,health &psychology,131,115,44,Put key data first. Customers start at the upp...


In [81]:
df_sn = pd.Series.to_frame(df["sessionnote"], name = "notes") #this makes series to dataframe

In [82]:
df_sn = df_sn.groupby(by = "notes").sum().reset_index() #unique sessionnotes total 89

In [83]:
df_sn

,notes
0,A reference to a full URL anyplace on a site o...
1,A site ought to have a viewable sign that a bu...
2,A user shouldnât need to tap on a link to ma...
3,Abstain from making dead-end pages on sites. T...
4,Adjust form labels and fields in a solitary ve...
...,...
84,When holding a tablet the sides and base of t...
85,When trying to find a search button on sites ...
86,When utilizing accordions offer an approach t...
87,While picking a site text font think about it...


In [84]:
#make sessionnote look index no. it finds the same element and takes its index no from df_sn
for i in range(len(df)):
    df["sessionnote"][i] = df_sn.where(df_sn == df["sessionnote"][i]).dropna().index[0]

In [86]:
df_ind = pd.Series.to_frame(df["industry"], name = "ind") #this makes series to dataframe

In [87]:
df_ind = df_ind.groupby(by = "ind").sum().reset_index()

In [88]:
df_ind.where(df_ind == df["industry"][0]).dropna().index[0]

4

In [89]:
df_ind

,ind
0,communication
1,e-commerce
2,education
3,energy&earth
4,entertainment
5,finance
6,food and beverage &consumption
7,health &psychology
8,tech
9,travel&transportation


In [90]:
for i in range(len(df)):
    df["industry"][i] = df_ind.where(df_ind == df["industry"][i]).dropna().index[0] 
    #find same industry elements in df_ind from df[indusry]. For example entertainment is in index 4 in df_ind then dropna for the nan tags 

In [91]:
 df["industry"]

0      4
1      7
2      1
3      9
4      4
      ..
133    3
134    8
135    3
136    7
137    3
Name: industry, Length: 138, dtype: object

In [120]:
df #latest look of the df table. Industry,tags and session notes are seen as their index number 

,data,industry,tag,tag2,tag3,sessionnote
0,how spotify organises work spotify recently ad...,4,95,128,-1,14
1,designing a video creation platform designing ...,7,110,148,-1,17
2,applying user research to a small e-commerce w...,1,154,-1,-1,64
3,democratizing access to bike maps cristiano sh...,9,110,20,-1,3
4,reimagine the future of television abdus share...,4,132,145,-1,79
...,...,...,...,...,...,...
133,rethinking an application against food waste a...,3,131,152,-1,16
134,the essential guide to enterprise login and si...,8,76,121,-1,65
135,uxui ecoai as responsible residents of van...,3,131,115,-1,32
136,medication disposal uxui design last week we f...,7,131,115,44,53


## EXTENSIVE DATA CLEANING PART

In [121]:
def textcln(synopsis):
    
    clean_text = synopsis
    
    while "\n" in clean_text:
        clean_text = clean_text.replace("\n"," ")
 
    #https://stackoverflow.com/questions/33404752/removing-emojis-from-a-string-in-python
    #https://en.wiktionary.org/wiki/▶ getting formation about the unicode of icon then remove that scale for extensive cleaning
    
    #however, I still see emoji and symbols in my csv file, therefore additional 
    #cleaning is done using regular expressions (EXTENSIVE CLEANING)
    unicode_pattern = re.compile("["
        u"\U00000080-\U000FFFFF"  # emoticons
        
                           "]+", flags=re.UNICODE)
    
    # no emoji and no blank by strip()
    clean_text= unicode_pattern.sub(r' ', clean_text).strip()
   
    
    clean_text=clean_text.lower()
    
    #there is no need to see ux case study, case study,Ux,UI case study texts on the file. IF the user enters these words, system cleans in the algorithm
    if "ux case study" in clean_text:
        clean_text= clean_text.replace("ux case study","")
    if "ux/ui case study" in clean_text:
        clean_text= clean_text.replace("UX/UI Case Study","")
    if "case study" in clean_text:
        clean_text= clean_text.replace("case study","")
    
    
    clean_text = "".join(u for u in clean_text if u not in ("&","#","?", ".", ";", ":", "!", "/", "|","(",")"))
    return clean_text


In [94]:
df["data"]=df["data"].apply(textcln)

In [95]:
df["sessionnote"] = df["sessionnote"].astype(int) 
df["data"] = df["data"].astype("string")

In [96]:
#adding tags at the end of the data(synopsis). tag2 and tag3 columns have some NAN cells therefore try and except helps with it
for i in range(len(df)):
    try:
        math.isnan(df_t1["tags"][i])  
    except:
        df["data"][i] = df["data"][i] + " " + df_t1["tags"][i]
    try:
        math.isnan(df_t2["tags"][i])    
    except:
        df["data"][i] = df["data"][i] + " " + df_t2["tags"][i]
    try:
        math.isnan(df_t3["tags"][i])
    except:
        df["data"][i] = df["data"][i] + " " + df_t3["tags"][i]
    

## Cosine Similarity CountVectorizer

In [97]:
def getSessionNote(df, synopsis, tag1, tag2, tag3):
    newEntry = synopsis + " " + tag1 +  " " + tag2 +  " " + tag3   #synopsis here means introduction and the title of the case study. 
    text = list(df['data'].values.astype('U')) + [newEntry]
    vect = CountVectorizer(stop_words='english') 
    #I fit the model with the words from the review text
    vect = vect.fit(text) 
    # make a matrix
    docu_feat = vect.transform(text) 
    cosine_sim = linear_kernel(docu_feat,docu_feat)
    scores = list(enumerate(cosine_sim[len(df)]))
    scores = sorted(scores, key = lambda x:x[1], reverse = True)
    scores = scores[1:3]
    indices = [i[0] for i in scores]
    return df_sn["notes"].iloc[list(df["sessionnote"].iloc[indices])]

## When the mentee enters her text (title, synopsis and tags), she will receieve suggestions according to cosine similarity

In [119]:
list(getSessionNote(df, "rethinking an application against consumption waste ", "sustainability", "nature", "waste"))

['The impression of site speed depends on load time, response time, load behavior and perfection of animations',
 'Just present one hint at a time on sites and mobile applications. This decreases memory burden']

In [99]:
df_ind

,ind
0,communication
1,e-commerce
2,education
3,energy&earth
4,entertainment
5,finance
6,food and beverage &consumption
7,health &psychology
8,tech
9,travel&transportation


## Creating a fake mentor list with 30 mentors with industry information attached to the mentors. As if mentors had selected their related industry when they created their profile

In [100]:
mentors = []   ### CREATING A SAMPLE LIST

for i in range(30):
    ind_count = random.randint(1,3) #how many counts 1,2,3
    industry_list = random.sample(range(len(df_ind)),ind_count) #I HAVE 10 INDUSTRY SO FAR
    mentors.append({"industry":[],"name":"mentor{}".format(i)})  #
    for industry_index in industry_list: #[5,4]
        mentors[i]["industry"].append(df_ind["ind"][industry_index])
       

In [101]:
mentors

[{'industry': ['finance'], 'name': 'mentor0'},
 {'industry': ['energy&earth', 'tech'], 'name': 'mentor1'},
 {'industry': ['food and beverage &consumption', 'entertainment'],
  'name': 'mentor2'},
 {'industry': ['entertainment', 'health &psychology'], 'name': 'mentor3'},
 {'industry': ['education'], 'name': 'mentor4'},
 {'industry': ['education'], 'name': 'mentor5'},
 {'industry': ['tech'], 'name': 'mentor6'},
 {'industry': ['food and beverage &consumption'], 'name': 'mentor7'},
 {'industry': ['e-commerce'], 'name': 'mentor8'},
 {'industry': ['entertainment', 'health &psychology', 'e-commerce'],
  'name': 'mentor9'},
 {'industry': ['energy&earth', 'health &psychology', 'e-commerce'],
  'name': 'mentor10'},
 {'industry': ['health &psychology', 'e-commerce'], 'name': 'mentor11'},
 {'industry': ['e-commerce', 'travel&transportation', 'health &psychology'],
  'name': 'mentor12'},
 {'industry': ['entertainment', 'e-commerce'], 'name': 'mentor13'},
 {'industry': ['travel&transportation'], 'na

# When the mentee enters the industry of the case study information, a mentor list with same industry will come as a match

In [102]:
def listMentors(industry):
    for mentor in mentors:
        if (industry in mentor["industry"]):
            print(mentor["name"])

In [103]:
listMentors("communication")

mentor19
mentor23


In [104]:
df

,data,industry,tag,tag2,tag3,sessionnote
0,how spotify organises work spotify recently ad...,4,95,128,-1,14
1,designing a video creation platform designing ...,7,110,148,-1,17
2,applying user research to a small e-commerce w...,1,154,-1,-1,64
3,democratizing access to bike maps cristiano sh...,9,110,20,-1,3
4,reimagine the future of television abdus share...,4,132,145,-1,79
...,...,...,...,...,...,...
133,rethinking an application against food waste a...,3,131,152,-1,16
134,the essential guide to enterprise login and si...,8,76,121,-1,65
135,uxui ecoai as responsible residents of van...,3,131,115,-1,32
136,medication disposal uxui design last week we f...,7,131,115,44,53


In [105]:
df_data = pd.Series.to_frame(df["data"], name = "data")


In [106]:
df.to_csv("data.csv",index = False)

In [107]:
df_ind.to_csv("industries.csv", index = False)

In [108]:
tag_inds = df_tags.join(pd.DataFrame({"industry":np.zeros(len(df_tags),dtype = np.int32)}))

In [109]:
tag_inds

,tags,industry
0,data visualization,0
1,AI,0
2,AR,0
3,HR,0
4,MacDonalds,0
...,...,...
151,vpn,0
152,waste,0
153,web page,0
154,website,0


In [110]:
for i in range(len(tag_inds)):
    tag_inds.at[i,"industry"] = df.query("tag == @i or tag2 == @i or tag3 == @i").reset_index()["industry"][0] 

In [111]:
tag_inds #industry has the index number instead of the industry name

,tags,industry
0,data visualization,0
1,AI,8
2,AR,8
3,HR,2
4,MacDonalds,6
...,...,...
151,vpn,8
152,waste,3
153,web page,8
154,website,1


In [112]:
tag_inds.to_csv("tags.csv", index = False)

In [113]:
df_sn=pd.read_csv("sessionnotes_clean.csv", encoding="utf_8_sig")

In [114]:
df_sessionnotes = df_sn.join(pd.DataFrame({"rating_design":np.zeros(len(df_sn),dtype = np.int32),
                                           "rating_uxresearch":np.zeros(len(df_sn),dtype = np.int32),
                                           "rating_writing":np.zeros(len(df_sn),dtype = np.int32),
                                           "rating_presentation":np.zeros(len(df_sn),dtype = np.int32),
                                           "isRated":np.zeros(len(df_sn),dtype = bool)}))

## I make a random rating to the UX tips that are suggested to the users, as if they have voted them before. I made 4 category to understand in which topic users can find the tips helpful. When user rates, the tip gains 1 point in that 4 category
Categories: DESIGN, UX RESEARCH, UX WRITING, PRESENTATION


In [115]:
#make a random rating to the UX tips 
for i in range(len(df_sessionnotes)):
    rate = random.randint(0,1)
    if(rate == 1):
        df_sessionnotes.at[i,"rating_design"] = random.randint(0,10)
        df_sessionnotes.at[i,"rating_uxresearch"] = random.randint(0,10)
        df_sessionnotes.at[i,"rating_writing"] = random.randint(0,10)
        df_sessionnotes.at[i,"rating_presentation"] = random.randint(0,10)
        df_sessionnotes.at[i,"isRated"] = True

In [116]:
df_sessionnotes.to_csv("sessionnotes.csv",encoding="utf_8_sig", index = False, sep = ";")

In [117]:
df_sessionnotes


,notes,rating_design,rating_uxresearch,rating_writing,rating_presentation,isRated
0,Consider the site as a red brick road: move yo...,8,10,4,7,True
1,Customers are more likely to see things close ...,0,0,0,0,False
2,Steady and simple to use web interfaces assist...,0,0,0,0,False
3,Abstain from making dead-end pages on sites. T...,0,0,0,0,False
4,Utilize common site patterns and interfaces; d...,0,0,0,0,False
...,...,...,...,...,...,...
99,Try not to USE ALL CAPS IN YOUR HEADLINES AND ...,9,6,0,5,True
100,NaN,0,0,0,0,False
101,NaN,0,0,0,0,False
102,\n\n\n\n,5,0,0,7,True


In [118]:
#creating a json fileimport json
with open("mentors.json","w") as outfile:
    json.dump(mentors, indent = 2, fp = outfile)